In [71]:
import json
import os
from dotenv import load_dotenv

load_dotenv()
path = os.getenv("FILE_PATH")
files_list = []

os.listdir(path)
print(os.listdir(path))


['Donghae-Dream', '망양_관광지.txt', '남창_관광지.txt', '오시리아_맛집.txt', '일광_맛집.txt', '서생_맛집.txt', '좌천_관광지.txt', '.DS_Store', '거제_관광지.txt', '센텀_맛집.txt', '개운포_맛집.txt', '동래_관광지.txt', 'Untitled-1.ipynb', '태화강_관광지.txt', '재송_관광지.txt', '월내_맛집.txt', '벡스코_관광지.txt', '거제해맞이_맛집.txt', '태화강_맛집.txt', '기장_맛집.txt', '동래_맛집.txt', '덕하_관광지.txt', '벡스코_맛집.txt', '부산원동_맛집.txt', '월내_관광지.txt', '부전역_관광지.txt', '신해운대_관광지.txt', '교대_관광지.txt', '오시리아_관광지.txt', '송정_관광지.txt', '부산원동_관광지.txt', '일광_관광지.txt', '재송_맛집.txt', '남창_맛집.txt', 'logo.png', '좌천_맛집.txt', 'AI', '교대_맛집.txt', '거제해맞이_관광지.txt', '안락_관광지.txt', '개운포_관광지.txt', '망양_맛집.txt', '안락_맛집.txt', '센텀_관광지.txt', '기장_관광지.txt', '부전역_맛집.txt', '신해운대_맛집.txt', 'chunker', '송정_맛집.txt', '서생_관광지.txt', '덕하_맛집.txt', '거제_맛집.txt']


In [72]:
import os

txt_files = [f for f in os.listdir(path) if f.endswith('.txt')]

print(txt_files)

['망양_관광지.txt', '남창_관광지.txt', '오시리아_맛집.txt', '일광_맛집.txt', '서생_맛집.txt', '좌천_관광지.txt', '거제_관광지.txt', '센텀_맛집.txt', '개운포_맛집.txt', '동래_관광지.txt', '태화강_관광지.txt', '재송_관광지.txt', '월내_맛집.txt', '벡스코_관광지.txt', '거제해맞이_맛집.txt', '태화강_맛집.txt', '기장_맛집.txt', '동래_맛집.txt', '덕하_관광지.txt', '벡스코_맛집.txt', '부산원동_맛집.txt', '월내_관광지.txt', '부전역_관광지.txt', '신해운대_관광지.txt', '교대_관광지.txt', '오시리아_관광지.txt', '송정_관광지.txt', '부산원동_관광지.txt', '일광_관광지.txt', '재송_맛집.txt', '남창_맛집.txt', '좌천_맛집.txt', '교대_맛집.txt', '거제해맞이_관광지.txt', '안락_관광지.txt', '개운포_관광지.txt', '망양_맛집.txt', '안락_맛집.txt', '센텀_관광지.txt', '기장_관광지.txt', '부전역_맛집.txt', '신해운대_맛집.txt', '송정_맛집.txt', '서생_관광지.txt', '덕하_맛집.txt', '거제_맛집.txt']


In [63]:
import re


def json_parser(file):
    print(file)
    with open(path+file) as f:
        file_text = f.read()
    objects = re.findall(r'\{.*?\}', file_text) # 리뷰를  {} 단위로 분리

    type(objects)
    # print(objects)

    # 원래 문자열
    obj_list:list = []
    # 작은따옴표를 큰따옴표로 변경
    for i in objects:
        # 작은따옴표를 큰따옴표로 변경
        obj = i.replace("'", '"')
        # JSON 파싱
        try:
            obj_list.append(json.loads(obj))
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")

    print(obj_list)
    return obj_list

In [64]:
from langchain_upstage import UpstageEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import CharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv()
gmap_key = os.getenv("GOOGLE_MAPS_API_KEY")

# 업스테이지 임베딩을 위한 커스텀 클래스 정의
embeddings = UpstageEmbeddings(
    api_key=gmap_key,
    model="solar-embedding-1-large"
)

#UpstageEmbeddings(model="solar-embedding-1-large-query") 쿼리 전용

from langchain.schema import Document
documents = []

def vector_db(obj_list):
    
    for obj in obj_list:
        # 'name'을 페이지 콘텐츠로, 'reviews'를 메타데이터로 사용
        metadata = {'name':obj['name'], 'lat':obj['lat'], 'lng':obj['lng']}
        content = str(obj['reviews'])
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)

        print(doc.metadata)

for i in txt_files:
    print(i)
    obj_list = json_parser(i)
    vector_db(obj_list)
    

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
# Chroma 벡터 데이터베이스 생성
db = Chroma.from_documents(docs, embeddings, persist_directory="db")



망양_관광지.txt
망양_관광지.txt
JSONDecodeError: Expecting ',' delimiter: line 1 column 185 (char 184)
[{'name': '태화루', 'lat': 35.553374, 'lng': 129.3077782, 'reviews': ['태화강과 국가정원이\n한눈에 내려다 보이는\n시원한 경치가 좋은 장소\n\n시티투어 막바지에 있어\n시간이 좀 남아서 방문했는데\n시원한 경치는 좋지만\n10분 정도 돌아보고 나면\n추가적으로 볼만한 게 없고\n\n여기서 중앙시장이나\n태화강국가정원으로 걸어가기에도\n애매한 위치\n\n궂이 일부러 찾아갈 정도는 아닌듯', '이곳의 태화강 뷰는 환상적입니다!\n저녁 노을이 질 때 이곳에 서서 바라보는 태화강은 평화의 미소 자체입니다!\n\n저는 이곳에 주로 산책을 하러 가곤 합니다.\n\n당신은 신발을 벗고 휴식을 취할 수도 있습니다!', '울산 태화강변의 역사적인 장소이네요. 오후 6시까지가 관람시간이라 올라가지는 못해도 둘러볼 수 있습니다', '새로 만든 누각이라 특별한 감동은 없지만 엄청나게 의미있는 곳으로 만들려는 정성은 충분히 느껴짐. 태화강 국가정원 쪽에서 멋진 일몰을 가장 편하게 볼 수 있는 장소.', '아주 좋은 곳이였어요! 넓고 볼거리도 많아서 즐거웠습니다. 태화루는 특히 주변과의 조화가 독특하고 아름다웠네요.']}, {'name': '장생포 고래문화마을', 'lat': 35.5058099, 'lng': 129.3826032, 'reviews': ['옛 풍경을 복원해서 어른들이 좋아할 만한 요소가 많습니다. 사진찍기 좋아요', '울산여행 마지막날\n수국축제한다고해서 구경왔는데이쁜곳도너무많고\n사진찍기도좋아요\n고래너무귀여워요.\n일하시는분들도 다 친절하시네요', '옛날 추억을 기억하게 합니다.\n아이들에게는 30-40년전의 한국의 모습을 알려줄 수 있습니다.\n6월에는 수국이 아름답습니다.', '수국 시즌이라 꼭 가보시길. 모노레일이 빨리 매진되니까 미리 예매하시

In [69]:
# 유사도 검색 예시
query = "웨이팅 30분인 곳 포장하면 1000원 이상 할인해주는 곳"
docs = db.similarity_search(query, k=5)
print(docs)
for d in docs:
    print(d.metadata['name'])



[Document(metadata={'lat': 35.19135539999999, 'lng': 129.0796237, 'name': '서울왕돈까스'}, page_content="['오픈 시간 맞춰 갔는데 30분 넘게 웨이팅 하고 들어갔습니다.\\n돈까스를 시킬 때 부어먹을지 찍어먹을지 선택 가능합니다.\\n일반적인 다른 돈까스에 비해 고기 두께, 부드러움, 맛 등 약간의 디테일은 더 좋았지만, 굳이 이렇게 기다리면서 먹어야 하나라는 생각은 많이 들지 않았습니다.\\n그래도 궁금하시면 한번 쯤은 먹으러 가도 좋은 집입니다!', '재재재재재방문 업데이트\\n\\n여전히 맛있고 친절하십니다\\n소박한 한끼가 멋진 식당입니다.\\n500미리 음료수 천원도 여전하고 뭔가 정성껏 식당을 꾸려나가시는 느낌입니다.\\n테이블 수가 작은데도 1인 고객도 식사 가능하며,\\n빈자리가 생기면 대기 순서대로 무작위로 테이블을 배정하십니다.\\n\\n3시부터 브레이크타임에서 =>3시 영업종료로 변경\\n6번 정도 방문해 본 결과 1시~1시 30분 사이 웨이팅이 길며 2시 이후 도착시 웨이팅 짧으나 늦어도 2시 15분까지는 도착해야 식사가 가능한 듯 합니다.\\n*치즈 돈까스는 1시 30분 이후 주문가능\\n\\n돈까스는 바삭하고 소스와도 조화롭습니다.\\n테이블 수가 많지 않지만 회전율은 빠른 편입니다. 일요일 휴무, 3시~4시 30분까지는 브레이크 타임이 있습니다.\\n\\n친절하고 합리적 가격에 무엇보다 맛있습니다.\\n요즘 물가에 500미리 패트병 음료수를 천원 받으셔서 놀랐습니다.  찍먹파 부먹파를 위해 돈까스 소스를 미리 얹을지 찍어 먹을지를 선택 할 수 있습니다.\\n찍먹 돈까스 소스와 부먹 소스가 맛이 다른데\\n이유를 안여쭤 봤네요.🙄\\n식전 스프, 깍두기 모두 한번에 갖다 주시지 않고 메인 음식이 나오기 전에 타이밍을 보고 가져다 주십니다.\\n수고스러우실텐데 뭔가 대접받는 느낌입니다.\\n남녀노소 무난히 좋아할 맛입니다.', '그냥 진짜 딱 옛날돈까스 그맛. 특별